In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'/Users/niralpatel/Desktop/Projects/classification_ETEP'

In [3]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/niral0901/classification_ETEP.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "niral0901"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "147e2b524c36b2df6a579a7e7a16b6b93ac5189b"

In [4]:
import tensorflow as tf

/Users/niralpatel/Desktop/Projects/classification_ETEP/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from Classsification_of_Kidney_Disease.constants import *
from Classsification_of_Kidney_Disease.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/niral0901/classification_ETEP.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        ) 
        return eval_config

In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import keras
keras_version = keras.__version__

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        keras_version = keras.__version__
        print(keras_version)
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model",  registered_model_name="VVG16")
            else:
                mlflow.keras.log_model(self.model, "model")

In [12]:
import keras

print(keras.__version__)


2.15.0


In [11]:
import tensorflow as tf
keras_version = tf.keras.__version__  # Accessing from within TensorFlow


AttributeError: module 'keras.api._v2.keras' has no attribute '__version__'

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-02-16 21:30:42,230: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-16 21:30:42,234: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-16 21:30:42,235: INFO: common: created directory at: artifacts]
Found 3733 images belonging to 1 classes.
234/234 [==============================] - 302s 1s/step - loss: 32.5768 - accuracy: 1.0000
[2024-02-16 21:35:45,221: INFO: common: json file saved at: scores.json]
2.15.0


2024/02/16 21:35:47 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/cn/59_k8cws6g95xx26rdq842z80000gn/T/tmp5af8up1h/model/data/model/assets
[2024-02-16 21:35:48,059: INFO: builder_impl: Assets written to: /var/folders/cn/59_k8cws6g95xx26rdq842z80000gn/T/tmp5af8up1h/model/data/model/assets]


Successfully registered model '1.0'.
2024/02/16 21:38:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: 1.0, version 1
Created version '1' of model '1.0'.


In [ ]:
print(tf.keras)


<module 'keras.api._v2.keras' from '/Users/niralpatel/Desktop/Projects/classification_ETEP/env/lib/python3.9/site-packages/keras/api/_v2/keras/__init__.py'>
